In [6]:
import json
import os
import time
import pandas as pd
import random
import time
import pickle

import numpy as np
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout

from keras.backend import clear_session
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

In [ ]:
temp = pd.read_csv('bb_all_comments.csv')
temp = temp.rename(index=str, columns={"You'll have to survive the mass hysteria and chaos first. People are going to come for what you have.": "comment"})
temp = temp.iloc[:,1:2]
temp.to_csv('bb_all_comments.csv')

In [ ]:
temp2 = pd.read_csv('bb_all_comments.csv')

In [7]:



bb_data = pd.read_csv('bb_all_comments.csv')
bb_data.comment=bb_data.comment.astype(str)
bb_data=bb_data['comment']


bb_data_lst = bb_data.tolist()

In [9]:
start = time.time()
bb_500k_tok = Tokenizer(num_words=50000,oov_token='<unk>')
bb_500k_tok.fit_on_texts(bb_data_lst[:500000])
with open('bb-500k-tok.pickle', 'wb') as handle:
    pickle.dump(bb_500k_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('bb tokenization time: {}'.format(time.time() - start))

bb tokenization time: 11.483397483825684


In [10]:
with open('bb-500k-tok.pickle', 'rb') as handle:
    bb_tok = pickle.load(handle)

In [11]:
len(bb_tok.word_index)

144151

In [12]:


train_data = bb_data_lst[:2000000]

seqs = bb_tok.texts_to_sequences(train_data)
less_20_seqs = list(filter(lambda x: len(x) <20, seqs))[:1000000]

In [13]:
MAXLEN = 19
VOCAB = int(len(bb_tok.word_index))
SEED = 42

sequences = pad_sequences(less_20_seqs, maxlen=MAXLEN, padding='pre')

print('Max Seq Len: %d' % MAXLEN)

sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.15, random_state=SEED)


#y_cat = to_categorical(y, num_classes=vocab_sz) #1-hot encoding 

Max Seq Len: 19


Need to optimize this later, use np arrays instead of python lists


For reference, seqs took 57s to go through 1 mill samples

In [14]:


#NN definition
model = Sequential()
model.add(Embedding(VOCAB, 300, input_length=MAXLEN-1))
model.add(LSTM(500))



model.add(Dense(VOCAB, activation='softmax'))
print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           43245300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 144151)            72219651  
Total params: 117,066,951
Trainable params: 117,066,951
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:

model_json = model.to_json()
with open("models/bb-model-v4.json", "w") as json_file:
    json_file.write(model_json)

fpath = "weights/bb-v4/best-weights.hdf5"
checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=2, save_best_only=True, mode='max')


BS = 100
tb = TensorBoard(log_dir="tensorboard-logs/{}".format('bb-modelv4'))
callback_lst = [checkpoint, tb]
#steps_per_epoch is num of batches that make up 1 epoch, defaults to size of train set
model.fit(X_train,y_train,batch_size=BS, validation_split=.005, epochs=200, callbacks=callback_lst, verbose=1)



Instructions for updating:
Use tf.cast instead.
Train on 845750 samples, validate on 4250 samples
Epoch 1/200
845750/845750 [==============================] - 999s 1ms/step - loss: 8.3093 - acc: 0.0625 - val_loss: 8.2482 - val_acc: 0.0661

Epoch 00001: val_acc improved from -inf to 0.06612, saving model to weights/bb-v4/best-weights.hdf5
Epoch 2/200
845750/845750 [==============================] - 998s 1ms/step - loss: 8.1928 - acc: 0.0627 - val_loss: 8.2200 - val_acc: 0.0661

Epoch 00002: val_acc did not improve from 0.06612
Epoch 3/200
845750/845750 [==============================] - 998s 1ms/step - loss: 8.1775 - acc: 0.0625 - val_loss: 8.1702 - val_acc: 0.0661

Epoch 00003: val_acc did not improve from 0.06612
Epoch 4/200
116500/845750 [===>..........................] - ETA: 14:21 - loss: 8.1851 - acc: 0.0628

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



395500/845750 [=============>................] - ETA: 8:49 - loss: 8.1687 - acc: 0.0632

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



609300/845750 [====================>.........] - ETA: 4:37 - loss: 8.1699 - acc: 0.0625

...

329700/845750 [==========>...................] - ETA: 10:06 - loss: 8.1648 - acc: 0.0616

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



580300/845750 [===================>..........] - ETA: 5:11 - loss: 8.1637 - acc: 0.0623

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



845750/845750 [==============================] - 994s 1ms/step - loss: 8.1677 - acc: 0.0624 - val_loss: 8.1970 - val_acc: 0.0661

Epoch 00016: val_acc did not improve from 0.06612
Epoch 17/200
845750/845750 [==============================] - 996s 1ms/step - loss: 8.1676 - acc: 0.0625 - val_loss: 8.1881 - val_acc: 0.0661

Epoch 00017: val_acc did not improve from 0.06612
Epoch 18/200
828200/845750 [============================>.] - ETA: 20s - loss: 8.1681 - acc: 0.0626